In [1]:
pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.9/626.9 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.7/485.7 KB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.6/700.6 KB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 KB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=c1e940f509c84c456b5055cc56c85a263228564531cf3570ff0d5ea6c196bf17
  Stored in directory: /root/.cache/pip/wheels/c

In [2]:
import torch.nn as nn
import torch
import math
import numpy as np
import pandas as pd
from numpy import linalg
from scipy.linalg import eigh
import torch.nn.functional as F
import random
# import wandb
from torch.nn.parameter import Parameter
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import scipy.sparse as sp
from torch import optim
import gc
import os.path as osp
import csv
from typing import Optional
from torch_geometric.typing import OptTensor
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import get_laplacian
from torch.autograd import Variable
from torch_geometric.nn import GCNConv, GATConv,RGCNConv
from torch_geometric.nn.inits import glorot, uniform
from torch_geometric.utils import softmax
from torch_sparse import SparseTensor, set_diag
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
from torch.nn import Sequential, Linear, ReLU, Dropout
from scipy.special import comb
from torch import Tensor
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv.gcn_conv import gcn_norm
import copy
import sklearn.metrics
import time
from sklearn.cluster import KMeans
from scipy.sparse import coo_matrix
from torch_geometric.nn.dense.linear import Linear as gLinear

In [3]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
#random.seed(42)
#np.random.seed(42)
#torch.manual_seed(42)
#torch.cuda.manual_seed(42)

cudaid = "cpu"
device = torch.device(cudaid)

root = r"/content/drive/MyDrive/data/GAPDA/"

In [5]:
piRNA = pd.read_csv(root + r"piRNA.csv", header = None).iloc[:,0].tolist()
disease = pd.read_csv(root + r"disease.csv", header = None, sep = "\t").iloc[:,0].tolist()
disease_feature = pd.read_csv(root + r"disease_feature.csv", header = None, dtype=np.float32).to_numpy()
piRNA_feature = pd.read_csv(root + r"piRNA_feature.csv", header = None, dtype=np.float32).to_numpy()
label = pd.read_csv(root + r"Label.csv", header = None, sep = "\t").iloc[:,0].tolist()
FeatureList = pd.read_csv(root + r"FeatureList.csv", header = None, sep = "\t", dtype="string")

In [6]:
def adj_normalization(w):
    m = w.shape[0]
    p = np.zeros([m, m])
    for i in range(m):
        for j in range(m):
            if i == j:
                p[i][j] = 1
            else:                
                p[i][j] = (w[i, j] - np.min(w[i, :])) / (np.max(w[i, :]) - np.min(w[i, :]))
    return p

In [7]:
disease_adj = adj_normalization(disease_feature)

In [8]:
ADJ = np.zeros((len(piRNA), len(disease)))
edge_idx_dict = {}
pos_1 = []
pos_2 = []
neg_1 = []
neg_2 = []
for i in range(len(label)):
    for _,m in FeatureList.iloc[i,:].items():
        m_list = m.split(",")
        if len(m_list)>2:            
            a = m_list[0]
            b = ",".join(m_list[1:],)
            m_list = []
            m_list.append(a)
            m_list.append(b)
        idx = piRNA.index(m_list[0])
        idy = disease.index(m_list[1])
        ADJ[idx, idy] = label[i]
        if label[i] > 0:
            pos_1.append(idx)
            pos_2.append(idy)
        else:
            neg_1.append(idx)
            neg_2.append(idy)
edge_idx_dict['pos_edges'] = np.vstack((pos_1, pos_2))
edge_idx_dict['neg_edges'] = np.vstack((neg_1, neg_2))

print(sum(sum(ADJ)))
print(ADJ.shape)
print(edge_idx_dict['pos_edges'].shape)
print(edge_idx_dict['neg_edges'].shape)
print(np.median(ADJ.sum(1)))
print(np.median(ADJ.sum(0)))

1212.0
(501, 22)
(2, 1212)
(2, 1212)
2.0
13.5


In [9]:
piRNA_sim = np.zeros((piRNA_feature.shape[0], piRNA_feature.shape[0]))
for i in range(piRNA_feature.shape[0]):
    for j in range(piRNA_feature.shape[0]):
        vec1 = piRNA_feature[i,:]
        vec2 = piRNA_feature[j,:]
        cos_sim = vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        piRNA_sim[i, j] = cos_sim

In [10]:
def GIP_kernel (Asso_RNA_Dis):
    # the number of row
    nc = Asso_RNA_Dis.shape[0]
    #initate a matrix as result matrix
    matrix = np.zeros((nc, nc))
    # calculate the down part of GIP fmulate
    r = getGosiR(Asso_RNA_Dis)
    #calculate the result matrix
    for i in range(nc):
        for j in range(nc):
            #calculate the up part of GIP formulate
            temp_up = np.square(np.linalg.norm(Asso_RNA_Dis[i,:] - Asso_RNA_Dis[j,:]))
            if r == 0:
                matrix[i][j]=0
            elif i==j:
                matrix[i][j] = 1
            else:
                matrix[i][j] = np.e**(-temp_up/r)
    return matrix
def getGosiR (Asso_RNA_Dis):
# calculate the r in GOsi Kerel
    nc = Asso_RNA_Dis.shape[0]
    summ = 0
    for i in range(nc):
        x_norm = np.linalg.norm(Asso_RNA_Dis[i,:])
        x_norm = np.square(x_norm)
        summ = summ + x_norm
    r = summ / nc
    return r

def read_txt1(path):
    with open(path, 'r', newline='') as txt_file:
        md_data = []
        reader = txt_file.readlines()
        for row in reader:
            line = row.split( )
            row = []
            for k in line:
                row.append(float(k))
            md_data.append(row)
        md_data = np.array(md_data)
        return md_data


#W is the matrix which needs to be normalized
def new_normalization (w):
    m = w.shape[0]
    p = np.zeros([m,m])
    for i in range(m):
        for j in range(m):
            if i == j:
                p[i][j] = 1/2
            elif np.sum(w[i,:])-w[i,i]>0:
                p[i][j] = w[i,j]/(2*(np.sum(w[i,:])-w[i,i]))
    return p

# get the KNN kernel, k is the number if first nearest neibors
def KNN_kernel (S, k):
    n = S.shape[0]
    S_knn = np.zeros([n,n])
    for i in range(n):
        sort_index = np.argsort(S[i,:])
        for j in sort_index[n-k:n]:
            if np.sum(S[i,sort_index[n-k:n]])>0:
                S_knn [i][j] = S[i][j] / (np.sum(S[i,sort_index[n-k:n]]))
    return S_knn


#updataing rules
def MiRNA_updating (S1,S2,S3,S4, P1,P2,P3,P4):
    it = 0
    P = (P1+P2+P3+P4)/4
    dif = 1
    while dif>0.0000001:
        it = it + 1
        P111 =np.dot (np.dot(S1,(P2+P3+P4)/3),S1.T)
        P111 = new_normalization(P111)
        P222 =np.dot (np.dot(S2,(P1+P3+P4)/3),S2.T)
        P222 = new_normalization(P222)
        P333 = np.dot (np.dot(S3,(P1+P2+P4)/3),S3.T)
        P333 = new_normalization(P333)
        P444 = np.dot(np.dot(S4,(P1+P2+P3)/3),S4.T)
        P444 = new_normalization(P444)
        P1 = P111
        P2 = P222
        P3 = P333
        P4 = P444
        P_New = (P1+P2+P3+P4)/4
        dif = np.linalg.norm(P_New-P)/np.linalg.norm(P)
        P = P_New
    print("Iter numb1", it)
    return P

def disease_updating(S1,S2, P1,P2):
    it = 0
    P = (P1+P2)/2
    dif = 1
    while dif> 0.0000001:
        it = it + 1
        P111 =np.dot (np.dot(S1,P2),S1.T)
        P111 = new_normalization(P111)
        P222 =np.dot (np.dot(S2,P1),S2.T)
        P222 = new_normalization(P222)
        P1 = P111
        P2 = P222
        P_New = (P1+P2)/2
        dif = np.linalg.norm(P_New-P)/np.linalg.norm(P)
        P = P_New
    print("Iter numb2", it)
    return P

def get_syn_sim (A, k1, k2):
    # disease_sim1 = read_txt1("./database/HMDD 2.0/disease_semantic_similarity.txt")
    # miRNA_sim1 = read_txt1("./database/HMDD 2.0/miRNA_functional_similarity.txt")
    # miRNA_sim2 = read_txt1 ("./database/HMDD 2.0/miRNA_sequence_similarity.txt")
    # miRNA_sim3 = read_txt1("./database/HMDD 2.0/miRNA_semantic_similarity.txt")

    disease_sim1 = read_txt1(root + r"disease_semantic_sim.txt")
    miRNA_sim1 = read_txt1(root + r"miRNA_functional_sim.txt")
    miRNA_sim2 = read_txt1 (root + r"miRNA_sequence_sim.txt")
    miRNA_sim3 = read_txt1(root + r"miRNA_semantic_sim.txt")

    GIP_m_sim = GIP_kernel(A)
    GIP_d_sim = GIP_kernel(A.T)
    #miRNA_sim1 = GIP_m_sim
    m1 = new_normalization(miRNA_sim1)
    # m1 = miRNA_sim1
    # m2 = miRNA_sim2
    # m3 = miRNA_sim3
    # m4 = GIP_m_sim
    m2 = new_normalization(miRNA_sim2)
    m3 = new_normalization(miRNA_sim3)
    m4 = new_normalization(GIP_m_sim)
    Sm_1 = KNN_kernel(miRNA_sim1, k1)
    Sm_2 = KNN_kernel(miRNA_sim2, k1)
    Sm_3 = KNN_kernel(miRNA_sim3, k1)
    Sm_4 = KNN_kernel(GIP_m_sim, k1)
    Pm = MiRNA_updating(Sm_1,Sm_2,Sm_3,Sm_4, m1, m2, m3,m4)
    Pm_final = (Pm + Pm.T)/2
    #np.save('m_sim_final.npy', Pm_final)
    #np.save('m_sim_final.txt', Pm_final)
    d1 = new_normalization(disease_sim1)
    d2 = new_normalization(GIP_d_sim)
    #d1 = disease_sim1
    #d2 = GIP_d_sim
    Sd_1 = KNN_kernel(disease_sim1, k2)
    Sd_2 = KNN_kernel(GIP_d_sim, k2)
    Pd = disease_updating(Sd_1,Sd_2, d1, d2)
    Pd_final = (Pd+Pd.T)/2
    #np.save('d_sim_final.npy', Pd_final)
    #np.save('d_sim_final.txt', Pd_final)

    return Pm_final, Pd_final

def get_all_the_samples(A):
    m,n = A.shape
    pos = []
    neg = []
    for i in range(m):
        for j in range(n):
            if A[i,j] ==1:
                pos.append([i,j,1])
            else:
                neg.append([i,j,0])
    n = len(pos)
    neg_new = random.sample(neg, n)
    tep_samples = pos + neg_new
    samples = random.sample(tep_samples, len(tep_samples))
    samples = random.sample(samples, len(samples))
    samples = np.array(samples)
    return samples

def get_all_the_samples1(A):
    m,n = A.shape
    pos = []
    neg = []
    for i in range(m):
        for j in range(n):
            if A[i,j] ==1:
                pos.append([i,j,1])
            else:
                neg.append([i,j,0])
    n = len(pos)
    neg_new = random.sample(neg, n)
    tep_samples = pos
    samples = random.sample(tep_samples, len(tep_samples))
    samples = random.sample(samples, len(samples))
    samples = np.array(samples)
    neg = np.array(neg)
    return samples, neg

def update_Adjacency_matrix (A, test_samples):
    m = test_samples.shape[0]
    A_tep = A.copy()
    for i in range(m):
        if test_samples[i,2] ==1:
            A_tep [test_samples[i,0], test_samples[i,1]] = 0
    return A_tep

def set_digo_zero(sim, z):
    sim_new = sim.copy()
    n = sim.shape[0]
    for i in range(n):
        sim_new[i][i] = z
    return sim_new

#get the sparse similarity matrix, k is the first k nearest neibors

def similarity_spare (A, k):
    m = A.shape[0]
    S = np.zeros([m,k])
    for i in range(m):
        tep = np.argsort(-A[i,:])
        S[i,0:k] = tep[0:k]

    W = np.zeros([m,m])
    for i in range(m):
        for j in range(m):
            if i in S[j,:] and j in S[i,:]:
                W[i,j] = 1
            elif i in S[j,:] or j in S[i,:]:
                W[i,j] = 0.5
            else:
                W[i,j] = 0
    A = np.multiply(A, W)
    return A


In [11]:
class PMF(object):
    def __init__(self, num_feat=16, epsilon=1, _lambda=0.1, momentum=0.8, maxepoch=200, num_batches=50, batch_size=10000):
        self.num_feat = num_feat  # Number of latent features,
        self.epsilon = epsilon  # learning rate,
        self._lambda = _lambda  # L2 regularization,
        self.momentum = momentum  # momentum of the gradient,
        self.maxepoch = maxepoch  # Number of epoch before stop,
        self.num_batches = num_batches  # Number of batches in each epoch (for SGD optimization),
        self.batch_size = batch_size  # Number of training samples used in each batches (for SGD optimization)

        self.w_Item = None  # Item feature vectors
        self.w_User = None  # User feature vectors

        self.rmse_train = []
        self.rmse_test = []

    # ***Fit the model with train_tuple and evaluate RMSE on both train and test data.  ***********#
    # ***************** train_vec=TrainData, test_vec=TestData*************#
    def fit(self, train_vec, test_vec):
        #torch.manual_seed(42)
        # mean subtraction
        self.mean_inv = np.mean(train_vec[:, 2])  # 评分平均值

        pairs_train = train_vec.shape[0]  # traindata 中条目数
        pairs_test = test_vec.shape[0]  # testdata中条目数

        # 1-p-i, 2-m-c
        num_user = int(max(np.amax(train_vec[:, 0]), np.amax(test_vec[:, 0]))) + 1  # 第0列，user总数
        num_item = int(max(np.amax(train_vec[:, 1]), np.amax(test_vec[:, 1]))) + 1  # 第1列，movie总数

        incremental = False  # 增量
        if ((not incremental) or (self.w_Item is None)):
            # initialize
            self.epoch = 0
            self.w_Item = 0.1 * np.random.randn(num_item, self.num_feat)  # numpy.random.randn 电影 M x D 正态分布矩阵
            self.w_User = 0.1 * np.random.randn(num_user, self.num_feat)  # numpy.random.randn 用户 N x D 正态分布矩阵

            self.w_Item_inc = np.zeros((num_item, self.num_feat))  # 创建电影 M x D 0矩阵
            self.w_User_inc = np.zeros((num_user, self.num_feat))  # 创建用户 N x D 0矩阵

        while self.epoch < self.maxepoch:  # 检查迭代次数
            self.epoch += 1

            # Shuffle training truples
            shuffled_order = np.arange(train_vec.shape[0])  # 根据记录数创建等差array
            np.random.shuffle(shuffled_order)  # 用于将一个列表中的元素打乱

            # Batch update
            for batch in range(self.num_batches):  # 每次迭代要使用的数据量
                # print "epoch %d batch %d" % (self.epoch, batch+1)

                test = np.arange(self.batch_size * batch, self.batch_size * (batch + 1))
                batch_idx = np.mod(test, shuffled_order.shape[0])  # 本次迭代要使用的索引下标

                batch_UserID = np.array(train_vec[shuffled_order[batch_idx], 0], dtype='int32')
                batch_ItemID = np.array(train_vec[shuffled_order[batch_idx], 1], dtype='int32')

                # Compute Objective Function
                pred_out = np.sum(np.multiply(self.w_User[batch_UserID, :],
                                              self.w_Item[batch_ItemID, :]),
                                  axis=1)  # mean_inv subtracted # np.multiply对应位置元素相乘

                rawErr = pred_out - train_vec[shuffled_order[batch_idx], 2] + self.mean_inv

                # Compute gradients
                Ix_User = 2 * np.multiply(rawErr[:, np.newaxis], self.w_Item[batch_ItemID, :]) \
                       + self._lambda * self.w_User[batch_UserID, :]
                Ix_Item = 2 * np.multiply(rawErr[:, np.newaxis], self.w_User[batch_UserID, :]) \
                       + self._lambda * (self.w_Item[batch_ItemID, :])  # np.newaxis :increase the dimension

                dw_Item = np.zeros((num_item, self.num_feat))
                dw_User = np.zeros((num_user, self.num_feat))

                # loop to aggreate the gradients of the same element
                for i in range(self.batch_size):
                    dw_Item[batch_ItemID[i], :] += Ix_Item[i, :]
                    dw_User[batch_UserID[i], :] += Ix_User[i, :]

                # Update with momentum
                self.w_Item_inc = self.momentum * self.w_Item_inc + self.epsilon * dw_Item / self.batch_size
                self.w_User_inc = self.momentum * self.w_User_inc + self.epsilon * dw_User / self.batch_size

                self.w_Item = self.w_Item - self.w_Item_inc
                self.w_User = self.w_User - self.w_User_inc

                # Compute Objective Function after
                if batch == self.num_batches - 1:
                    pred_out = np.sum(np.multiply(self.w_User[np.array(train_vec[:, 0], dtype='int32'), :],
                                                  self.w_Item[np.array(train_vec[:, 1], dtype='int32'), :]),
                                      axis=1)  # mean_inv subtracted
                    rawErr = pred_out - train_vec[:, 2] + self.mean_inv
                    obj = np.linalg.norm(rawErr) ** 2 \
                          + 0.5 * self._lambda * (np.linalg.norm(self.w_User) ** 2 + np.linalg.norm(self.w_Item) ** 2)

                    self.rmse_train.append(np.sqrt(obj / pairs_train))

                # Compute validation error
                if batch == self.num_batches - 1:
                    pred_out = np.sum(np.multiply(self.w_User[np.array(test_vec[:, 0], dtype='int32'), :],
                                                  self.w_Item[np.array(test_vec[:, 1], dtype='int32'), :]),
                                      axis=1)  # mean_inv subtracted
                    rawErr = pred_out - test_vec[:, 2] + self.mean_inv
                    self.rmse_test.append(np.linalg.norm(rawErr) / np.sqrt(pairs_test))

                    # Print info
                    #if batch == self.num_batches - 1:
                    #    print('Training RMSE: %f, Test RMSE %f' % (self.rmse_train[-1], self.rmse_test[-1]))
        return self.w_Item, self.w_User

    def predict(self, invID):
        return np.dot(self.w_Item, self.w_User[int(invID), :]) + self.mean_inv  # numpy.dot 点乘

    # ****************Set parameters by providing a parameter dictionary.  ***********#
    def set_params(self, parameters):
        if isinstance(parameters, dict):
            self.num_feat = parameters.get("num_feat", 10)
            self.epsilon = parameters.get("epsilon", 1)
            self._lambda = parameters.get("_lambda", 0.1)
            self.momentum = parameters.get("momentum", 0.8)
            self.maxepoch = parameters.get("maxepoch", 20)
            self.num_batches = parameters.get("num_batches", 10)
            self.batch_size = parameters.get("batch_size", 1000)

    def topK(self, test_vec, k=10):
        inv_lst = np.unique(test_vec[:, 0])
        pred = {}
        for inv in inv_lst:
            if pred.get(inv, None) is None:
                pred[inv] = np.argsort(self.predict(inv))[-k:]  # numpy.argsort索引排序

        intersection_cnt = {}
        for i in range(test_vec.shape[0]):
            if test_vec[i, 1] in pred[test_vec[i, 0]]:
                intersection_cnt[test_vec[i, 0]] = intersection_cnt.get(test_vec[i, 0], 0) + 1
        invPairs_cnt = np.bincount(np.array(test_vec[:, 0], dtype='int32'))

        precision_acc = 0.0
        recall_acc = 0.0
        for inv in inv_lst:
            precision_acc += intersection_cnt.get(inv, 0) / float(k)
            recall_acc += intersection_cnt.get(inv, 0) / float(invPairs_cnt[int(inv)])

        return precision_acc / len(inv_lst), recall_acc / len(inv_lst)

In [12]:
def cheby(i,x):
    if i==0:
        return 1
    elif i==1:
        return 2*x
    else:
        T0=1
        T1=2*x
        for ii in range(2,i+1):
            T2=2*x*T1-T0
            T0,T1=T1,T2
        return T2

In [13]:
class ChebnetII_prop(MessagePassing):
    def __init__(self, K, Init=False, bias=True, **kwargs):
        super(ChebnetII_prop, self).__init__(aggr='add', **kwargs)
        
        self.K = K
        self.temp = Parameter(torch.Tensor(self.K+1))
        self.Init = Init
        self.reset_parameters()

    def reset_parameters(self):
        self.temp.data.fill_(1.0)

        if self.Init:
            for j in range(self.K+1):
                x_j = math.cos((self.K-j+0.5)*math.pi/(self.K+1))
                self.temp.data[j] = x_j**2
        
    def forward(self, x, edge_index,edge_weight=None):
        coe_tmp = F.relu(self.temp)
        coe = coe_tmp.clone()
        
        for i in range(self.K + 1):
            coe[i] = coe_tmp[0]*cheby(i,math.cos((self.K + 0.5)*math.pi/(self.K + 1)))
            for j in range(1,self.K + 1):
                x_j = math.cos((self.K - j + 0.5)*math.pi/(self.K + 1))
                coe[i] = coe[i] + coe_tmp[j]*cheby(i, x_j)
            coe[i] = 2*coe[i]/(self.K + 1)


        #L=I-D^(-0.5)AD^(-0.5)
        edge_index1, norm1 = get_laplacian(edge_index, edge_weight,normalization='sym', dtype=x.dtype, num_nodes=x.size(self.node_dim))

        #L_tilde=L-I
        edge_index_tilde, norm_tilde = add_self_loops(edge_index1,norm1,fill_value=-1.0,num_nodes=x.size(self.node_dim))

        Tx_0 = x
        Tx_1 = self.propagate(edge_index_tilde,x=x,norm=norm_tilde,size=None)

        out = coe[0]/2*Tx_0+coe[1]*Tx_1

        for i in range(2,self.K+1):
            Tx_2=self.propagate(edge_index_tilde,x=Tx_1,norm=norm_tilde,size=None)
            Tx_2=2*Tx_2-Tx_0
            out=out+coe[i]*Tx_2
            Tx_0,Tx_1 = Tx_1, Tx_2
        return out


    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def __repr__(self):
        return '{}(K={}, temp={})'.format(self.__class__.__name__, self.K,
                                          self.temp)



In [14]:

class VGAEModel2(nn.Module):
    def __init__(self, num_features, hidden, out_dim, K, dropout = 0.5, device = 'cpu'):
        super(VGAEModel2, self).__init__()        
        self.num_features = num_features
        self.hidden = hidden
        self.out_dim = out_dim
        self.K = K
        self.dropout = dropout
        self.device = device

        self.lin1 = nn.Linear(num_features, hidden)
        self.lin2 = nn.Linear(hidden, out_dim)

        layers = [ChebnetII_prop(K), ChebnetII_prop(K), ChebnetII_prop(K//2)]
        self.layers = nn.ModuleList(layers)
        

    def encoder(self, features, edge_index, adj):       
        #x, edge_index

        x = F.dropout(features, p = self.dropout, training=self.training)
        x = self.lin1(x)
        x = F.relu(x)        

        x = F.dropout(x, p = self.dropout, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.relu(self.layers[0](x, edge_index))
        
        #x = F.dropout(x, p=self.dropout, training=self.training)
        #h = torch.relu(self.layers[0](x, edge_index))
        #print(h.shape)
        #h = t.relu(self.layers[1](h, edge_index, adj_tensor[i][edge_index[0], edge_index[1]]))
        #self.mean = self.layers[1](h, edge_index)
        #self.mean = t.relu(self.layers[1](h, edge_index))*0.01
        #self.log_std = self.layers[2](h, edge_index)

        self.mean = self.layers[1](x, edge_index)
        self.log_std = self.layers[1](x, edge_index)

        gaussian_noise = torch.randn(features.size(0), self.mean.shape[1]).to(device)
        
        #pl = t.poisson(self.mean)
            
        #print("-----------------------gaussian_noise.shape{}".format(gaussian_noise.shape))
        #print("-----------------------self.log_std.shape{}".format(self.log_std.shape))
            
        sampled_z = self.mean + gaussian_noise * torch.exp(self.log_std).to(self.device)
        return sampled_z
        #return pl

    def decoder(self, z):
        adj_rec = torch.sigmoid(torch.mm(z, z.T))
        return adj_rec

    def forward(self, features, edge_index, adj):
        z = self.encoder(features, edge_index, adj)
        adj_rec = self.decoder(z)
        return z,adj_rec, self.mean, self.log_std
        #return z,adj_rec, self.mean

In [15]:

def get_edge_index(matrix):
    edge_index = [[], []]
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i, j] != 0:
                edge_index[0].append(i)
                edge_index[1].append(j)
    return torch.LongTensor(edge_index)

In [16]:
class BuildModel3(nn.Module):  
 
    def __init__(self, num_i, num_h1, num_h2, num_o, alpha = 0.5, dropout = 0.5):
        super(BuildModel3,self).__init__()

        self.alpha = alpha
        
        self.linear11 = nn.Linear(num_i,num_h1)
        self.relu11 = nn.ReLU()
        self.linear12 = nn.Linear(num_i,num_h1)
        self.relu12 = nn.ReLU()
        
        self.linear21 = nn.Linear(num_h1,num_h2) #2个隐层
        self.relu21 = nn.ReLU()
        self.linear22 = nn.Linear(num_h1,num_h2) #2个隐层
        self.relu22 = nn.ReLU()
        
        self.linear31 = nn.Linear(num_h2,num_o)
        self.linear32 = nn.Linear(num_h2,num_o)
        
        self.dropout = nn.Dropout(dropout)
        self.num_i = num_i
  
    def forward(self, x):
        temp = x 
        x = temp[:,:32]
        x = self.dropout(x)
        x = self.linear11(x)        
        x = self.dropout(x)
        x = self.relu11(x)
        x = self.linear21(x)        
        x = self.dropout(x)
        x = self.relu21(x)
        x1 = self.linear31(x)
        
        x = temp[:,32:]
        x = self.dropout(x)
        x = self.linear12(x)        
        x = self.dropout(x)
        x = self.relu12(x)
        x = self.linear22(x)        
        x = self.dropout(x)
        x = self.relu22(x)
        x2 = self.linear32(x)        
        
        x = self.alpha*x1 + (1 - self.alpha)*x2
        
        return torch.sigmoid(x)


In [17]:
def train(model, adj_label_m, features_m, optimizer, epochs, norm):
    #torch.manual_seed(42)
    model.train()
    edge_index = get_edge_index(adj_label_m)

    adj_label_m_t = torch.Tensor(adj_label_m)
    features_m_t = torch.Tensor(features_m)
    pos_weight = []

    for i in adj_label_m_t.reshape([-1]):
        if i >= 0.5:
            pos_weight.append(50)
        else:
            pos_weight.append(1)
    regression_crit3 = nn.BCEWithLogitsLoss(weight = torch.Tensor(pos_weight))


    for epoch in range(1, epochs + 1):
        model.zero_grad()
        score, adj_rec, vgae_mean, vgae_std = model(features_m_t, edge_index, adj_label_m_t)        
        loss = norm * regression_crit3(adj_rec.reshape([-1]), adj_label_m_t.reshape([-1]))      
        
        logits = adj_rec
        kl_divergence = (0.5 / logits.shape[0]) * (1 + 2 * vgae_std - 
                        vgae_mean ** 2 - torch.exp(vgae_std) ** 2).sum(1).mean()
        
        loss -= kl_divergence       
        
        regression_crit3.weight = torch.Tensor(pos_weight)*torch.exp(-(2**0.5)*abs(adj_rec.reshape([-1]).detach() - 
                              adj_label_m_t.reshape([-1]).detach()))

        loss = loss.requires_grad_()        
        loss.backward()
        optimizer.step()            
        #print("0000")
        if epoch%100 == 0:
            print("epoch : %d, loss:%.4f" % (epoch, loss))
    return score

In [18]:
def Feature_synthesis_2(feature_MFm, feature_MFd, feature_m_Z, feature_d_Z, train_samples, test_samples):
    n1, n2 = feature_MFm.shape[1], feature_MFd.shape[1] 
    n3, n4 = feature_m_Z.shape[1], feature_d_Z.shape[1]
    Feature_dim = n1 + n2 + n3 + n4    

    train_n = train_samples.shape[0]
    train_feature = np.zeros([train_n, Feature_dim])
    
    train_label = np.zeros([train_n])
    for i in range(train_n):
        train_feature[i,0:n1] = feature_MFm[train_samples[i,0],:]
        train_feature[i,n1 :(n1+n2)] = feature_MFd[train_samples[i,1], :]
        train_feature[i,(n1+n2):(n1+n2+n3)] = feature_m_Z.detach()[train_samples[i,0],:]
        train_feature[i, (n1+n2+n3):(n1+n2+n3+n4)] = feature_d_Z.detach()[train_samples[i,1],:]        
        train_label[i] = train_samples[i,2]


    # get the featrue vectors of test samples
    test_N = test_samples.shape[0]
    test_feature = np.zeros([test_N, Feature_dim])
    
    test_label = np.zeros(test_N)
    for i in range(test_N):
        test_feature[i,0:n1] = feature_MFm[test_samples[i,0],:]
        test_feature[i,n1 :(n1+n2)] = feature_MFd[test_samples[i,1], :]
        test_feature[i,(n1+n2):(n1+n2+n3)] = feature_m_Z.detach()[test_samples[i,0],:]
        test_feature[i, (n1+n2+n3):(n1+n2+n3+n4)] = feature_d_Z.detach()[test_samples[i,1],:]        
        test_label[i] = test_samples[i,2]
    return train_feature, train_label, test_feature, test_label

In [19]:
def skf_normalization(w):
    row_sum = np.sum(w, axis=0)
    p = (w / row_sum).T
    return p


def skf(A, seq_sim, str_sim, k1, k2):
    disease_sim1 = str_sim
    circRNA_sim1 = seq_sim

    GIP_c_sim = GIP_kernel(A)
    GIP_d_sim = GIP_kernel(A.T)
    # miRNA_sim1 = GIP_m_sim
    m1 = skf_normalization(circRNA_sim1)
    m2 = skf_normalization(GIP_c_sim)

    Sm_1 = KNN_kernel(circRNA_sim1, k1)
    Sm_2 = KNN_kernel(GIP_c_sim, k1)
    Pm = skf_updating(Sm_1, Sm_2, m1, m2, 0.1)
    nei_weight1 = neighborhood_Com(Pm, k1)
    Pm_final = Pm * nei_weight1

    d1 = skf_normalization(disease_sim1)
    d2 = skf_normalization(GIP_d_sim)

    Sd_1 = KNN_kernel(disease_sim1, k2)
    Sd_2 = KNN_kernel(GIP_d_sim, k2)
    Pd = skf_updating(Sd_1, Sd_2, d1, d2, 0.1)
    nei_weight2 = neighborhood_Com(Pd, k2)
    Pd_final = Pd * nei_weight2

    return Pm_final, Pd_final


def skf_updating(S1, S2, P1, P2, alpha):
    P = (P1 + P2) / 2
    dif = 1
    while dif > 0.0000001:
        P111 = alpha * np.dot(np.dot(S1, P2), S1.T) + (1 - alpha) * P2
        P111 = new_normalization(P111)
        P222 = alpha * np.dot(np.dot(S2, P1), S2.T) + (1 - alpha) * P1
        P222 = new_normalization(P222)
        P1 = P111
        P2 = P222
        P_New = (P1 + P2) / 2
        dif = np.linalg.norm(P_New - P) / np.linalg.norm(P)
        P = P_New
    return P


def neighborhood_Com(sim, k):
    weight = np.zeros(sim.shape)

    for i in range(sim.shape[0]):
        iu = sim[i, :]
        iu_list = np.abs(np.sort(-iu))
        iu_nearest_list_end = iu_list[k - 1]
        for j in range(sim.shape[1]):
            ju = sim[:, j]
            ju_list = np.abs(np.sort(-ju))
            ju_nearest_list_end = ju_list[k - 1]

            if sim[i, j] >= iu_nearest_list_end and sim[i, j] >= ju_nearest_list_end:
                weight[i, j] = 1
                weight[j, i] = 1
            elif sim[i, j] < iu_nearest_list_end and sim[i, j] < ju_nearest_list_end:
                weight[i, j] = 0
                weight[j, i] = 0
            else:
                weight[i, j] = 0.5
                weight[j, i] = 0.5

    return weight

In [20]:
def k_matrix(matrix, k=20):
    num = matrix.shape[0]
    knn_graph = np.zeros(matrix.shape)
    idx_sort = np.argsort(-(matrix - np.eye(num)), axis=1)
    for i in range(num):
        #将第i行最大的前k个值赋值给knn_graph(确保是对称矩阵)
        knn_graph[i, idx_sort[i, :k + 1]] = matrix[i, idx_sort[i, :k + 1]]
        knn_graph[idx_sort[i, :k + 1], i] = matrix[idx_sort[i, :k + 1], i]
    return knn_graph + np.eye(num)


In [21]:
from sklearn.metrics import precision_recall_curve,roc_curve, accuracy_score,f1_score,auc,precision_score,recall_score,matthews_corrcoef
def cross_validation_experiment_3(samples, A, k_fold = 5, num_feat = 16,
                  last_dim_m = 16, last_dim_d = 16, K_m = 16, K_d = 16, k1= 50, k2 = 5,
                  dropout = 0.5, device = device, epochs = 1000, pos_weight = 50, norm = 0.5):
    #A = miRNA_dis_matrix
    kf = KFold(n_splits = k_fold, shuffle = True)    
    metric = np.zeros((1, 7))
    pre_matrix = np.zeros(A.shape)    
    torch.manual_seed(42)
    train_index, test_index = [],[]
    for i, j in kf.split(samples):
        train_index.append(i)
        test_index.append(j)
    auc_avg = 0
    f1_avg = 0
    aupr_avg = 0
    mcc_avg = 0
    acc_avg = 0
    pre_avg = 0
    rec_avg = 0
    
    for k in range(k_fold):
        print("------this is %dth cross validation------" % (k + 1))
        train_samples = samples[train_index[k], :]
        test_samples = samples[test_index[k], :]                
        
        pmf = None
        pmf = PMF(num_feat = num_feat, maxepoch=200)
        feature_MFd, feature_MFm = pmf.fit(train_samples, test_samples)        
        #train_samples = samples[train_index[0], :]
        #test_samples = samples[test_index[0], :]

        new_A = update_Adjacency_matrix(A, test_samples)
        #feature_MFm, feature_MFd = low_svd(new_A, meta_dim = 22)
        features_m = new_A
        features_d = new_A.transpose()
        sim_m, sim_d = skf(new_A, piRNA_sim, disease_adj, k1, k2)
        
        m_adj = k_matrix(sim_m, k = 25)
        d_adj = k_matrix(sim_d, k = 10)
        
        model = None
        model = VGAEModel2(new_A.shape[1], 3*last_dim_m, last_dim_m, K = K_m, dropout = dropout, device = device)        
        optimizer = None
        optimizer = optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.001)
        feature_m_Z = train(model, m_adj, features_m, optimizer, epochs, norm)
        model = None
        model = VGAEModel2(new_A.shape[0], 3*last_dim_d, last_dim_d, K = K_d, dropout = dropout, device = device)        
        optimizer = None
        optimizer = optim.Adam(model.parameters(), lr = 0.0001, weight_decay = 0.001)
        feature_d_Z = train(model, d_adj, features_d, optimizer, epochs, norm)
        
        train_feature, train_label, test_feature, test_label = Feature_synthesis_2(feature_MFm, feature_MFd,                                                                  
                                                  feature_m_Z, feature_d_Z,
                                                  train_samples, test_samples)       
        
        modelMLP = BuildModel3(32, 16, 8, 1, 0.9, dropout)        
        train_label = torch.Tensor(train_label)
        
        regression_critMLP = nn.BCEWithLogitsLoss()        
        optimizer_MLP = optim.Adam(modelMLP.parameters(), lr = 0.001, weight_decay = 0.0001)

        train_feature = torch.Tensor(train_feature)

        for i in range(150):
            modelMLP.zero_grad()           
            PRE = modelMLP(train_feature)       
    
            loss = regression_critMLP(PRE.view(1,-1), train_label.view(1,-1))           
        
            loss = loss.requires_grad_()           
            loss.backward()
            optimizer_MLP.step()
            if i%50 == 0:
                print(loss)

        modelMLP.eval()
        test_label = torch.Tensor(test_label)
        test_feature = torch.Tensor(test_feature)
        PRE_test = modelMLP(test_feature)
        y_true = test_label.view(1,-1).numpy().flatten()  
        y_score = PRE_test.view(1,-1).detach().numpy().flatten()
        fpr, tpr, thresholds = roc_curve(y_true, y_score)
        precision, recall, _ = precision_recall_curve(y_true, y_score)
        auroc=auc(fpr, tpr)
        aupr=auc(recall, precision)
        acc=accuracy_score(y_true, np.rint(y_score))
        F1=f1_score(y_true, np.rint(y_score), average='macro')
        Pre=precision_score(y_true, np.rint(y_score), average='macro')
        Rec=recall_score(y_true, np.rint(y_score), average='macro')        
        
        Mcc=matthews_corrcoef(y_true,np.rint(y_score))        
        print('AUC',auroc)
        print('AUPR',aupr)
        print('ACC',acc)
        print('F1-score',F1)
        print('Precision',Pre)
        print('Recall',Rec)
        print('MCC',Mcc)

        auc_avg += auroc
        aupr_avg += aupr
        acc_avg += acc
        f1_avg += F1
        pre_avg += Pre
        rec_avg += Rec
        mcc_avg += Mcc

    print("auc_avg: {}, aupr_avg: {}, acc_avg: {}, f1_avg: {}, pre_avg: {}, rec_avg: {}, mcc_avg: {}".format(auc_avg/k_fold, aupr_avg/k_fold, acc_avg/k_fold, f1_avg/k_fold, pre_avg/k_fold, rec_avg/k_fold, mcc_avg/k_fold))
    return PRE_test, test_label      




In [22]:
A = ADJ
samples = get_all_the_samples(A)

In [27]:
PRE_test, test_label = cross_validation_experiment_3 (samples, A, k_fold = 5, num_feat = 16,
                  last_dim_m = 16, last_dim_d = 16, K_m = 2, K_d = 2, k1= 51, k2 = 5,
                  dropout = 0.01, device = device, epochs = 1000, pos_weight = 50, norm = 0.5)

------this is 1th cross validation------
epoch : 100, loss:0.3055
epoch : 200, loss:0.3010
epoch : 300, loss:0.2996
epoch : 400, loss:0.2999
epoch : 500, loss:0.3013
epoch : 600, loss:0.2999
epoch : 700, loss:0.2993
epoch : 800, loss:0.2981
epoch : 900, loss:0.2995
epoch : 1000, loss:0.2993
epoch : 100, loss:0.6716
epoch : 200, loss:0.6500
epoch : 300, loss:0.6583
epoch : 400, loss:0.6530
epoch : 500, loss:0.6325
epoch : 600, loss:0.6396
epoch : 700, loss:0.6197
epoch : 800, loss:0.6260
epoch : 900, loss:0.6328
epoch : 1000, loss:0.6191
tensor(0.7280, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.7161, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.6798, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
AUC 0.922506577373834
AUPR 0.9045174027883576
ACC 0.8144329896907216
F1-score 0.8134615384615385
Precision 0.8368504469850768
Recall 0.8231540643045068
MCC 0.6598623826405544
------this is 2th cross validation------
epoch : 100, loss:0.3421
epoch : 200, loss:0.3104

In [24]:
240436/1939

124.0

In [25]:
32+60

92

In [26]:
"""
0.5
0.9107069987013399, aupr_avg: 0.9010285909670287, acc_avg: 0.8391113572463149, f1_avg: 0.8374643421649439, pre_avg: 0.8536387107358342, rec_avg: 0.8393448450560868, mcc_avg: 0.6928254313401777
0.9  30
0.913414992210841, aupr_avg: 0.8906125343785469, acc_avg: 0.8440598108545625, f1_avg: 0.8429268457860639, pre_avg: 0.8525443053545988, rec_avg: 0.8442178431163455, mcc_avg: 0.6966983739413687
0.9  16_all
0.9120714532002582, aupr_avg: 0.8959739679809801, acc_avg: 0.8106355968305359, f1_avg: 0.80659707617568, pre_avg: 0.8392021563954521, rec_avg: 0.8107835933162157, mcc_avg: 0.6493539014261125
0.9  16_all dropout=0.6
auc_avg: 0.9136362056152942, aupr_avg: 0.8901365312181572, acc_avg: 0.808587373264037, f1_avg: 0.8042759455295014, pre_avg: 0.8374207664158988, rec_avg: 0.808704727861774, mcc_avg: 0.6454623628470884



"""

'\n0.5\n0.9107069987013399, aupr_avg: 0.9010285909670287, acc_avg: 0.8391113572463149, f1_avg: 0.8374643421649439, pre_avg: 0.8536387107358342, rec_avg: 0.8393448450560868, mcc_avg: 0.6928254313401777\n0.9  30\n0.913414992210841, aupr_avg: 0.8906125343785469, acc_avg: 0.8440598108545625, f1_avg: 0.8429268457860639, pre_avg: 0.8525443053545988, rec_avg: 0.8442178431163455, mcc_avg: 0.6966983739413687\n0.9  16_all\n0.9120714532002582, aupr_avg: 0.8959739679809801, acc_avg: 0.8106355968305359, f1_avg: 0.80659707617568, pre_avg: 0.8392021563954521, rec_avg: 0.8107835933162157, mcc_avg: 0.6493539014261125\n0.9  16_all dropout=0.6\nauc_avg: 0.9136362056152942, aupr_avg: 0.8901365312181572, acc_avg: 0.808587373264037, f1_avg: 0.8042759455295014, pre_avg: 0.8374207664158988, rec_avg: 0.808704727861774, mcc_avg: 0.6454623628470884\n\n\n\n'